1. Классификация

In [15]:
import pandas as pd
import glob
import os
import tqdm
import gc
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns
import numpy as np
from scipy.spatial import Voronoi, voronoi_plot_2d
import math
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score, confusion_matrix
from tqdm import tqdm
%matplotlib inline

def get_data():
    f='C:\\Users\\anna.nazarova\\Downloads\\Telegram Desktop\\athletes_sochi.txt'
    dfs = pd.read_csv(f).dropna()
    dfs["BMI"] = dfs["weight"] / (dfs["height"]**2)
    alldata = dfs[ ['age', 'weight', 'height', 'BMI', 'gender'] ]
    alldata = alldata.sample(frac=1)
    allinput = alldata[ ['age', 'weight', 'height', 'BMI'] ]
    alloutput = alldata["gender"].apply(lambda x: int(x == "Male"))
    return alldata, allinput, alloutput

def predict(alldata, allinput, alloutput):
    chunk_80 = int(len(alldata) * 0.8)
    X_train = allinput[ :chunk_80]
    Y_train = alloutput[ :chunk_80]
    X_test = allinput[chunk_80: ]
    Y_test = alloutput[chunk_80: ]
    model = linear_model.LogisticRegression()
    model.fit(X_train, Y_train)
    Y_test_predicted = model.predict(X_test)
    print_accuracy(Y_test, Y_test_predicted)
    
def print_accuracy(Y_test, Y_test_predicted):
    [tn, fp], [fn, tp] = confusion_matrix(Y_test, Y_test_predicted)
    accuracy = (tn + tp) / (tp + fp + tn + fn)
    print(f"Accuracy: {accuracy}")
    return accuracy

alldata, allinput, alloutput = get_data()
predict(alldata, allinput, alloutput)

Accuracy: 0.8810483870967742


2. Кластеризация (разделить страны на кластеры по уровню развития; метод k-means)

In [11]:
# Импортируем библиотеки
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import pandas as pd


# Загружаем набор данных и масштабируем
def get_data():
    global country_df
    country_df = pd.read_csv("C:\\Users\\anna.nazarova\\Desktop\\Мага\\Матан\\ДЗ_2_Кластеризация\\Country-data.csv")
    dataset = country_df.drop(['country'], axis =1) 
    columns = dataset.columns
    scaler = MinMaxScaler()
    rescaled_dataset_minmax = scaler.fit_transform(dataset) 
    df_minmax = pd.DataFrame(data=rescaled_dataset_minmax , columns = columns)
    return df_minmax

# Кластеризация данных методом k-means; планирую разделением стран на 3 кластера: развитые, развивающиеся, отсталые страны.
def k_means(df_minmax):
    model = KMeans (n_clusters = 3) 
    y_predicted_minmax = model.fit_predict(df_minmax) 
    country_df['cluster'] = y_predicted_minmax
    clusters_table = pd.pivot_table(country_df, index=['cluster']) #средние значения для каждого признака по кластерам
    print(clusters_table, '\n')
    #выделяем страны, входящие в одну группу
    cluster_0 = country_df.loc[country_df['cluster'] == 0]
    print(cluster_0.country.unique(), '\n')
    cluster_1 = country_df.loc[country_df['cluster'] == 1]
    print(cluster_1.country.unique(), '\n')
    cluster_2 = country_df.loc[country_df['cluster'] == 2]
    print(cluster_2.country.unique(), '\n')
    
df_minmax = get_data()
k_means(df_minmax)

         child_mort    exports          gdpp    health    imports  \
cluster                                                             
0         93.284783  29.287174   1695.913043  6.338478  43.297826   
1         22.425581  40.382430   6719.790698  6.215581  46.932162   
2          4.897143  58.431429  43117.142857  8.917429  51.508571   

               income  inflation  life_expec  total_fer  
cluster                                                  
0         3516.804348  12.097065   59.393478   5.090217  
1        12770.813953   7.609023   72.582558   2.293256  
2        45802.857143   2.535000   80.245714   1.741143   

['Afghanistan' 'Angola' 'Benin' 'Burkina Faso' 'Burundi' 'Cameroon'
 'Central African Republic' 'Chad' 'Comoros' 'Congo, Dem. Rep.'
 'Congo, Rep.' "Cote d'Ivoire" 'Equatorial Guinea' 'Eritrea' 'Gabon'
 'Gambia' 'Ghana' 'Guinea' 'Guinea-Bissau' 'Haiti' 'Iraq' 'Kenya'
 'Kiribati' 'Lao' 'Lesotho' 'Liberia' 'Madagascar' 'Malawi' 'Mali'
 'Mauritania' 'Mozambique' '